# [Classify Text into Labels](https://python.langchain.com/docs/tutorials/classification/)

# ■ with_structured_outputを使う方法

- [モデルから構造かデータを返す方法](https://python.langchain.com/docs/how_to/structured_output/)

In [9]:
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticToolsParser


# プロンプト
tagging_prompt = """
以下の文章から必要な情報を抽出してください。
'Classification' 機能に存在するプロパティのみを抽出してください。

文章:
{input}
"""

prompt = ChatPromptTemplate([
    HumanMessagePromptTemplate.from_template(tagging_prompt)
])


# 出力のデータ構造
class Classification(BaseModel):
    sentiment: str = Field(
        ...,
        description="文章の感情",
        enum=["positive", "neutral", "negative"]
    )
    aggressiveness: int = Field(
        ...,
        description="攻撃的な度合。数字が大きいほど攻撃的",
        enum=[1, 2, 3, 4, 5],
    )
    language: str = Field(
        ...,
        enum=["Japanese", "English", "Other"]
    )


llm = ChatOpenAI(model="gpt-4o", temperature=0).with_structured_output(Classification)

tagging_chain = prompt | llm


In [10]:
input = """
家族のスマートフォン充電のために購入
Ankerということもあり品質は十分
充電速度は24WなのでPDには対応しておりません。

なので、寝る前に充電する方におすすめ
他にもUSBで充電する機器などでUSB充電器が足りない方にもおすすめ

値段も手軽で電源端子も折りたたみ収納できますので旅行にも持っていきやすいです。
"""

tagging_chain.invoke({"input": input})

Classification(sentiment='positive', aggressiveness=1, language='Japanese')

In [11]:
input = """
仕事用と趣味用に2台購入して２年ほど経ちました。使用頻度の低い趣味用の方でコントロールキーとNキーが反応しないことが頻繁に起きるようになりました。掃除してみようとキーをバラしたところスペースキーが下の部品まで折れてくっついてきて元に戻せなくなりご臨終しました。
また打鍵感は爪に負担を感じ、指が痛くなることが多々ありました。
見た目やバックライトは綺麗で気に入っていたので残念です。
"""

tagging_chain.invoke({"input": input})

Classification(sentiment='negative', aggressiveness=2, language='Japanese')

In [12]:
input = """
使い始めて約2年ほど経ったあたりでキーの反応が悪くなる現象が出始めた。
1日1～2時間ライトに使ってただけなので耐久性は非常に低いと言わざるを得ない。
他の方のレビューでも書かれているようなので品質は低いのでしょう。

初めてのゲーミングキーボードで1万3千円もしたのに2年持たなかったのはガッカリしました。
ハイパーXのキーボードはもう買いませんし、お勧めしません。
"""
tagging_chain.invoke({"input": input})

Classification(sentiment='negative', aggressiveness=3, language='Japanese')

# ■ シンプルにOutputParserをチェインにつなげる方法

In [5]:
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser

#
# プロンプト
#
tagging_prompt = """
以下の文章から必要な情報を抽出してください。

{format_instructions}

文章:
{input}
"""

prompt = ChatPromptTemplate([
    HumanMessagePromptTemplate.from_template(tagging_prompt)
])


#
# 出力パーサー
#

# 出力データ構造
#   ...: 必須フィールド
#   enum: 各プロパティが取りうる値のリスト
#   description: モデルがプロパティの意味を理解するための説明
class Classification(BaseModel):
    sentiment: str = Field(
        ...,
        description="文章の感情",
        enum=["positive", "neutral", "negative"]
    )
    aggressiveness: int = Field(
        ...,
        description="攻撃的な度合。数字が大きいほど攻撃的",
        enum=[1, 2, 3, 4, 5],
    )
    language: str = Field(
        ...,
        enum=["Japanese", "English", "Other"]
    )

parser = PydanticOutputParser(pydantic_object=Classification)


#
# モデル
#
llm = ChatOpenAI(model="gpt-4o", temperature=0)

#
# チェーン
#
tagging_chain = prompt | llm | parser

In [6]:
input = """
家族のスマートフォン充電のために購入
Ankerということもあり品質は十分
充電速度は24WなのでPDには対応しておりません。

なので、寝る前に充電する方におすすめ
他にもUSBで充電する機器などでUSB充電器が足りない方にもおすすめ

値段も手軽で電源端子も折りたたみ収納できますので旅行にも持っていきやすいです。
"""

tagging_chain.invoke({"input": input, "format_instructions": parser.get_format_instructions()})

Classification(sentiment='positive', aggressiveness=1, language='Japanese')

In [7]:
input = """
仕事用と趣味用に2台購入して２年ほど経ちました。使用頻度の低い趣味用の方でコントロールキーとNキーが反応しないことが頻繁に起きるようになりました。掃除してみようとキーをバラしたところスペースキーが下の部品まで折れてくっついてきて元に戻せなくなりご臨終しました。
また打鍵感は爪に負担を感じ、指が痛くなることが多々ありました。
見た目やバックライトは綺麗で気に入っていたので残念です。
"""

tagging_chain.invoke({"input": input, "format_instructions": parser.get_format_instructions()})

Classification(sentiment='negative', aggressiveness=2, language='Japanese')

In [8]:
input = """
使い始めて約2年ほど経ったあたりでキーの反応が悪くなる現象が出始めた。
1日1～2時間ライトに使ってただけなので耐久性は非常に低いと言わざるを得ない。
他の方のレビューでも書かれているようなので品質は低いのでしょう。

初めてのゲーミングキーボードで1万3千円もしたのに2年持たなかったのはガッカリしました。
ハイパーXのキーボードはもう買いませんし、お勧めしません。
"""
tagging_chain.invoke({"input": input, "format_instructions": parser.get_format_instructions()})

Classification(sentiment='negative', aggressiveness=3, language='Japanese')